# Automatic Field Booking

### Imports

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # Keys in the keyboard
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from datetime import datetime
from datetime import timedelta
import sys

### Input Parameters

In [ ]:
Wanted_parks = ["Regents"] # For now it only works specifying 1 single park, later I want to make it book to different parks based on user preferences
wanted_time_slots = ["09:00","16:00"] # format --> A list of start slot times in the following format: ["07:00","16:00"]  Order the list based on your preferred slot, the preferred slots go first

card_num = input("Insert card numbers: format xxxxxxxxxxxxxxxx    ")
card_exp = input("insert card expiry: format xxxx    ")
card_cvc = input("Insert card cvc: format xxx    ")

Card_Details = [card_num,card_exp,card_cvc]

Parks = {"Hyde":"https://www.openplay.co.uk/booking/place/4038/member-choice?mode=nonMember","Regents":"https://www.openplay.co.uk/booking/place/154/member-choice?mode=nonMember"}

''' This is the day for which we want to book. Since Will To Win let's you book 3 days from today,
we shoud select 4 
days from today. We will make the booking at midnight, as soon as the new shedule is out '''
the_new_day = (datetime.today() + timedelta(days=4)).strftime('%Y-%m-%d')

In [ ]:
chromedriver = "chromedriver.exe"

os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = Options()

# Turn off chrome warning for multiple downloads
# prefs = {'profile.default_content_setting_values.automatic_downloads': 1,"download.default_directory" : download_folder_path}
# chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
wanted_park_1 = Wanted_parks[0]
driver.get(Parks[wanted_park_1])

### Selecting the Game: Padel

In [ ]:
driver.find_element_by_xpath("//select[@name='use']/option[text()='Padel']").click()

### Selecting the date

In [ ]:
def time_until_end_of_day(dt=None):
    if dt is None:
        dt = datetime.now()
    return ((24 - dt.hour - 1) * 60 * 60) + ((60 - dt.minute - 1) * 60) + (60 - dt.second)

seconds_till_midnight = time_until_end_of_day()
time.sleep(seconds_till_midnight) # Wait untill midnight and then execute the following code

Park_table_xpaths = {"Hyde":"//table[@class='table  table-striped table-times resource-3546']/tbody","Regents":"//table[@class='table  table-striped table-times resource-3530']/tbody"}

number_of_iterations = 0
# 1440 iterazioni corrispondono a 24h ore (1 min a iterazione)
all_options_list = []
is_new_day_flag = False
while (number_of_iterations < 1440) or (is_new_day_flag == True):
    # Get Possible selections
    select_box = driver.find_element_by_xpath("//select[@name='date']") 
    options = [x for x in select_box.find_elements_by_tag_name("option")]
    options_vals = [elem.get_attribute("value") for elem in options]
    now = datetime.now()
    number_of_iterations+=1
    ##############################################
    latest_date = options_vals[-1]
    formatted_ = "//select[@name='date']/option[@value='{}']".format(latest_date)
    driver.find_element_by_xpath(formatted_).click()
    
    table = driver.find_element_by_xpath(Park_table_xpaths[wanted_park_1])
    row_list = table.find_elements_by_tag_name("tr")
    row_list = row_list[1:] #Removing the header row
    Time_slots = []
    for row in row_list:
        time_slot = row.find_element_by_tag_name("td").get_attribute("class")
        try:
            avaiable = row.find_element_by_tag_name("td").find_element_by_tag_name("a").get_attribute("href")
        except:
            avaiable = "Not avaiable"
        Time_slots.append((time_slot,avaiable))
    avaiable_time_slots = [ i for i in Time_slots if i[1] != "Not avaiable"]
    time_options_avaiable = (now, latest_date,avaiable_time_slots)
    all_options_list.append(time_options_avaiable)
    is_new_day_flag = (latest_date == the_new_day) #Check if the new_day schedule is out
    
    if is_new_day_flag == True: # If it is out, we save the new schedule and we break the loop
        new_day_schedule = time_options_avaiable
        break
    time.sleep(60) # Stop Execution for 1 minute
    driver.refresh() # Refresh page
    
    
    

### Checking if any of the slots avaiable in the new schedule are desired from us

In [ ]:
new_day_time_slots_avaiable = new_day_schedule[2] # List of tuples with tuple[0] = 'text-center slot-3546 07:00'

times_avaiable = []
for tup in new_day_time_slots_avaiable:
    time_ = tup[0][-5:]
    times_avaiable.append(time_)

selected_time = False
for wanted_time in wanted_time_slots:
    if wanted_time in times_avaiable:
        selected_time = wanted_time
        break

if selected_time == False:
    print("You wanted one of the following time slots {}".format(wanted_time_slots))
    print("Only the following time slots where avaiable for the day {} : {}".format(latest_date,times_avaiable))
    print("Too bad, if will be for another time")
    exit("Too bad, if will be for another time - Exit program")
    
else:
    print("You selected time slot {}, which was avaiable".format(wanted_time))
    print("We will now process your booking")

### Select the time

In [ ]:
# mod the format to "07:00-" Starting time specified
time.sleep(1)
starting_time = selected_time + "-"
driver.find_element_by_partial_link_text(starting_time).click()

### Select Pricing

In [ ]:
time.sleep(2)
# General Public
iframe = driver.find_element_by_xpath("//iframe[@name='smallModal-frame']")
driver.switch_to.frame(iframe)
driver.find_element_by_partial_link_text("General Public").click()
driver.switch_to.default_content()


In [ ]:
# Click Continue
time.sleep(2)
driver.find_element_by_id("cart-continue").click()

In [ ]:
# Existing User
time.sleep(1)
driver.find_element_by_partial_link_text("Existing User").click()

In [ ]:
#Input username and password
time.sleep(1)
user_username = "toardo@gmail.com"
user_password = "I_Reserve_Fast_6"

elem_username = driver.find_element_by_id("loginEmail")
elem_username.clear()
elem_username.send_keys(user_username)



elem_password = driver.find_element_by_id("loginPassword")
elem_password.clear()
elem_password.send_keys(user_password)
elem_password.send_keys(Keys.RETURN)

In [ ]:
time.sleep(1)
driver.find_element_by_id("submit").click()

In [ ]:
time.sleep(1)
driver.find_element_by_id("continue").click()

In [ ]:
time.sleep(1)
driver.find_element_by_id("confirm-checkbox").click()

In [ ]:
time.sleep(1)
driver.find_element_by_id("complete-order").click()

In [ ]:
# smallModal-frame is the one we want
time.sleep(3)
first_layer_iframe = {child_frame.get_attribute('name'):child_frame for child_frame in driver.find_elements_by_tag_name('iframe')}
selected_iframe = first_layer_iframe['smallModal-frame']

In [ ]:
driver.switch_to.frame(selected_iframe)

In [ ]:
time.sleep(3)
second_layer_iframe = {child_frame.get_attribute('name'):child_frame for child_frame in driver.find_elements_by_tag_name('iframe')}

# Keep only the ones with "privateStripeFrame" in them
second_layer_iframe_2 = {key:elem for key,elem in second_layer_iframe.items() if "privateStripeFrame" in key}
#These 3 are Card number - Expiry and CVC in this order.

second_layer_iframe_sorted = sorted(second_layer_iframe_2.items())  

In [ ]:
card_iframe = second_layer_iframe_sorted[0][1]
Expiry_iframe = second_layer_iframe_sorted[1][1]
CVC_iframe = second_layer_iframe_sorted[2][1]

In [ ]:
# connect to Card Iframe
driver.switch_to.frame(card_iframe)
time.sleep(2)
inputs_list = driver.find_elements_by_xpath("//input")
inputs_list[1].send_keys(Card_Details[0]) #Send CARD numbers
time.sleep(1)

# Switch to Default Frame
driver.switch_to.default_content()
time.sleep(3)

# Connect to Stripe Frame
driver.switch_to.frame(selected_iframe)
time.sleep(3)
# Connect to Expiry Frame
driver.switch_to.frame(Expiry_iframe) 

inputs_list = driver.find_elements_by_xpath("//input")
inputs_list[1].send_keys(Card_Details[1]) #Send CARD expiry
time.sleep(1)

# Switch to Default Frame
driver.switch_to.default_content()
time.sleep(3)

# Connect to Stripe Frame
driver.switch_to.frame(selected_iframe)
time.sleep(3)

# Connect to CVC Frame
driver.switch_to.frame(CVC_iframe) 
inputs_list = driver.find_elements_by_xpath("//input")
inputs_list[1].send_keys(Card_Details[2]) #Send CVC expiry
inputs_list[1].send_keys(Keys.RETURN)
time.sleep(1)